In [1]:
import torch

import matplotlib.pyplot as plt
import torchmetrics as tm

from config import *
from stages import *
from train import *
from data.ssa import SSA

from data.util import crop_q_between, split_weekdays_and_weekends

%reload_ext autoreload
%autoreload 2

In [38]:
mse = tm.MeanSquaredError().to(CONFIG.device)
mape = tm.MeanAbsolutePercentageError().to(CONFIG.device)
mat_q = CONFIG.load('mat_q.pt')
CONFIG.alpha = 0.057
CONFIG.device
mat_q

tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  4.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  1., 16.,  ..., 29.,  0.,  0.],
        [ 0.,  0.,  9.,  ..., 16.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]], device='cuda:0')

##### Compare MSE and MAPE losses for different parameter configurations

Pre-train DBN and Train DBN attaching KELM on each epoch

In [41]:
results_wd = {}
results_we = {}

# ssa = SSA(CONFIG.time_window_length // 3, 36, CONFIG.device)
ssa = SSA(4, [2, 2], CONFIG.device)

CONFIG.spectral_threshold = 15440
CONFIG.dbn_hidden_layer_sizes = [84, 84, 84]

gamma = 1
reg_coeff = 26

for P in [3125,]:#range(3125 * 3, 28125, 3125):
    CONFIG.spectral_threshold = P
    for N in [324,]:#range(4, 245, 80):
      CONFIG.dbn_hidden_layer_sizes = [N, N, N]
      dbn = DBN(CONFIG.time_window_length, CONFIG.dbn_hidden_layer_sizes,
                CONFIG.gibbs_sampling_steps).to(CONFIG.device)
      mat_q_trend, mat_q_resid = preprocess_data(CONFIG.spectral_threshold, mat_q)
      mat_c, mat_x, nonempty, representatives = compress_data(
          mat_q_resid.abs(), CONFIG.read_period, CONFIG.train_period, CONFIG.alpha)
      print(mat_c.shape)
      mat_q_trend = mat_q_trend[:, nonempty][:, representatives].abs()
    #   mat_q_resid = mat_q_resid[:, nonempty].real

      (_, val_trend_wd_dataloader, _), (_, val_trend_we_dataloader, _) = crop_and_split_mat(mat_q_trend, CONFIG)
      # (_, val_resid_wd_dataloader, _), (_, val_resid_we_dataloader, _) = crop_and_split_mat(mat_q_resid, CONFIG)
      mat_c_wd_datasets, mat_c_we_datasets = split_mat(mat_c, CONFIG)

      del mat_c

      dbn, kelm, val_resid_we_dataloader = train_with_config(CONFIG, dbn, mat_c_we_datasets, dbn_training_epochs=100, stride=1, gamma=gamma, reg_coeff=reg_coeff)

      mse_loss = torch.tensor([0.,]).to(CONFIG.device)
      iter_trend = iter(val_trend_we_dataloader)
      n_samples = 0
      for X_resid, y_resid in val_resid_we_dataloader:
          X_trend, y_trend = next(iter_trend)
          pred_trend = ssa.forecast(X_trend.squeeze().T, 1).sum(0)[-1][None]

          pred_resid = dbn(X_resid).squeeze()
          pred_resid = kelm(pred_resid).T

          pred = pred_trend + pred_resid

          y = y_trend + y_resid

          mse_loss += mse(pred, y)
          n_samples += 1

      mse_loss /= n_samples

      print(f'gamma={gamma}, reg_coeff={reg_coeff}, WE P={CONFIG.spectral_threshold}, N={CONFIG.dbn_hidden_layer_sizes}, Loss={mse_loss.item()}')

    # N = results_wd[P]
    # data = torch.tensor(tuple(results_wd[P].items()))
    # plt.suptitle(f'WD P={P}')
    # plt.xlabel('N')
    # plt.ylabel('Loss')
    # plt.plot(data[:, 0], data[:, 1])
    # plt.show()

    # N = results_we[P]
    # data = torch.tensor(tuple(results_we[P].items()))
    # plt.suptitle(f'WE P={P}')
    # plt.xlabel('N')
    # plt.ylabel('Loss')
    # plt.plot(data[:, 0], data[:, 1])
    # plt.show()

torch.Size([2016, 2])


100%|██████████| 100/100 [02:29<00:00,  1.50s/it]


gamma=1, reg_coeff=26, WE P=1440, N=[80, 80, 80], Loss=505.2121887207031


KeyboardInterrupt: 

In [33]:
iter_trend = iter(val_trend_we_dataloader)
mse_loss = 0
n_samples = 0
for X_resid, y_resid in val_resid_we_dataloader:
    X_trend, y_trend = next(iter_trend)
    pred_trend = ssa.forecast(X_trend.squeeze().T, 1).sum(0)[-1][None]

    pred_resid = dbn(X_resid).squeeze()
    pred_resid = kelm(pred_resid).T

    pred = pred_trend + pred_resid

    y = y_trend + y_resid

    mse_loss += mse(pred, y)
    n_samples += 1

mse_loss /= n_samples

print(f'gamma={gamma}, reg_coeff={reg_coeff}, WE P={CONFIG.spectral_threshold}, N={CONFIG.dbn_hidden_layer_sizes}, Loss={mse_loss.item()}')

KeyboardInterrupt: 